### Задание  
Взять датасет который прикреплён по ссылке https://www.dropbox.com/s/ykqk49a8avlmnaf/ru_all_split.tar.gz  
обучить модель которая будет генерировать заголовки к постам

### Чтение датасета

In [1]:
from datasets import load_dataset

In [2]:
data_files = {"train": "./data/ru_all_train.jsonl",
              "validation": "./data/ru_all_val.jsonl",
              "test": "./data/ru_all_test.jsonl"}
dataset = load_dataset("json", data_files=data_files)

Using custom data configuration default-10569b3db47d67a1
Reusing dataset json (C:\Users\Kartsev.ES\.cache\huggingface\datasets\json\default-10569b3db47d67a1\0.0.0\da492aad5680612e4028e7f6ddc04b1dfcec4b64db470ed7cc5f2bb265b9b6b5)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'text', 'url', 'timestamp'],
        num_rows: 616216
    })
    validation: Dataset({
        features: ['title', 'text', 'url', 'timestamp'],
        num_rows: 34234
    })
    test: Dataset({
        features: ['title', 'text', 'url', 'timestamp'],
        num_rows: 34235
    })
})

In [4]:
dataset['train']['title'][0]

'В разных районах Омска появились свои "Эйфелевы башни"'

In [5]:
dataset['train']['text'][0]

'В городе опоры ЛЭП стали необычно подсвечиваться. \n         Омичи рассказали в соцсетях о необычном способе украсить ночной город: теперь опоры ЛЭП на улицах Ипподромной, 8-й Линии и Масленникова, а также в районе городка Водников подсвечиваются разными цветами. Фотографии опубликовали в группе "Омск Live". В комментариях часть омичей скептически отнеслась к такому световому оформлению и отметила, что лучше бы эти средства пустили на освещение городских магистралей или улиц в частном секторе. - И никто не заметил, что в частном секторе ни одного фонаря вообще, в лесу и то проще найти выход, чем в частном секторе Омска ночью,  - пишут омичи. Некоторые пользователи сравнили омские ЛЭП с Эйфелевой башней, так как они теперь тоже подсвечиваются и переливаются с наступлением тёмного времени суток. Отметим, что символ Франции уже украшал омские улицы. Один из них находится в Советском округе в районе Первомайского рынка. Изначально пятиметровая башня была установлена в Выставочном сквере, 

In [6]:
def len_tok(text):
    return len(text.split())

In [7]:
max_len_txt, max_len_tl = max(map(len_tok, dataset['train']['text'])), max(map(len_tok, dataset['train']['title']))
max_len_txt, max_len_tl

(11026, 151)

### Токенизация

In [8]:
max_len_txt, max_len_tl = 100, 25
model_name = "IlyaGusev/rut5_base_sum_gazeta"

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    tokenized_input = tokenizer(batch['text'], padding='max_length', truncation=True, max_length=max_len_txt)
    tokenized_label = tokenizer(batch['title'], padding='max_length', truncation=True, max_length=max_len_tl)

    tokenized_input['labels'] = tokenized_label['input_ids']

    return tokenized_input

dataset_train = dataset['train'].shuffle(seed=42).select(range(1000)).map(tokenize, batched=True, batch_size=8)
dataset_val = dataset['validation'].shuffle(seed=42).select(range(100)).map(tokenize, batched=True, batch_size=8)
dataset_test = dataset['test'].shuffle(seed=42).select(range(100)).map(tokenize, batched=True, batch_size=8)

dataset_train.set_format('numpy', columns=['input_ids', 'attention_mask', 'labels'])
dataset_val.set_format('numpy', columns=['input_ids', 'attention_mask', 'labels'])
dataset_test.set_format('numpy', columns=['input_ids', 'attention_mask', 'labels'])

Loading cached shuffled indices for dataset at C:\Users\Kartsev.ES\.cache\huggingface\datasets\json\default-10569b3db47d67a1\0.0.0\da492aad5680612e4028e7f6ddc04b1dfcec4b64db470ed7cc5f2bb265b9b6b5\cache-a8b9446d3198fe2f.arrow
Parameter 'function'=<function tokenize at 0x000002478707BC10> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/125 [00:00<?, ?ba/s]

Loading cached shuffled indices for dataset at C:\Users\Kartsev.ES\.cache\huggingface\datasets\json\default-10569b3db47d67a1\0.0.0\da492aad5680612e4028e7f6ddc04b1dfcec4b64db470ed7cc5f2bb265b9b6b5\cache-d7573fe11dc98030.arrow


  0%|          | 0/13 [00:00<?, ?ba/s]

Loading cached shuffled indices for dataset at C:\Users\Kartsev.ES\.cache\huggingface\datasets\json\default-10569b3db47d67a1\0.0.0\da492aad5680612e4028e7f6ddc04b1dfcec4b64db470ed7cc5f2bb265b9b6b5\cache-1c4d366bb9a9ddbc.arrow


  0%|          | 0/13 [00:00<?, ?ba/s]

In [10]:
dataset_train.save_to_disk('./data/ru_all/train')
dataset_val.save_to_disk('./data/ru_all/validation')
dataset_test.save_to_disk('./data/ru_all/test')

### Обучение

In [11]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments

In [12]:
model = T5ForConditionalGeneration.from_pretrained(model_name)

In [13]:
for param in model.encoder.parameters():
    param.requires_grad = False

for param in model.decoder.parameters():
    param.requires_grad = False

In [14]:
output_dir = 'ru_all/output'

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_accumulation_steps=1, # Number of eval steps to keep in GPU (the higher, the mor vRAM used)
    prediction_loss_only=True, # If I need co compute only loss and not other metrics, setting this to true will use less RAM
    learning_rate=0.00001,
    evaluation_strategy='steps', # Run evaluation every eval_steps
    save_steps=1000, # How often to save a checkpoint
    save_total_limit=1, # Number of maximum checkpoints to save
    remove_unused_columns=True, # Removes useless columns from the dataset
    run_name='run_gazeta', # Wandb run name
    logging_steps=500, # How often to log loss to wandb
    eval_steps=500, # How often to run evaluation on the val_set
    logging_first_step=False, # Whether to log also the very first training step to wandb
    load_best_model_at_end=True, # Whether to load the best model found at each evaluation.
    metric_for_best_model="loss", # Use loss to evaluate best model.
    greater_is_better=False # Best model is the one with the lowest loss, not highest.
)

In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_val
)

trainer.train()

The following columns in the training set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: timestamp, text, url, title. If timestamp, text, url, title are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
C:\Users\Kartsev.ES\Anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 375
C:\Users\Kartsev.ES\Anaconda3\lib\site-packages\transformers\data\data_collator.py:130: UserWarning: Creating a tensor from a list of nump

Step,Training Loss,Validation Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=375, training_loss=13.289520833333333, metrics={'train_runtime': 759.4579, 'train_samples_per_second': 3.95, 'train_steps_per_second': 0.494, 'total_flos': 398283264000000.0, 'train_loss': 13.289520833333333, 'epoch': 3.0})

### Генерация заголовков

In [16]:
INX = 50
print("TEXT: | {}".format(dataset_test['text'][INX]))
print("TITLE: | {}".format(dataset_test['title'][INX]))

TEXT: | 29 мая правоохранители задержали при получении взятки руководителя одного из медицинских учреждений  Львова . Об этом  сообщает  пресс-служба СБУ. Медик обещал «договориться» с руководством районного военного комиссариата об освобождении от прохождения срочной военной службы по состоянию здоровья. Свои услуги он оценил в 21,5 тысяч гривен. В кабинете заведующего отделением также обнаружили наличные в разной валюте: 150 тысяч гривен, 600 долларов и 400 евро. Правоохранители не разглашают название отделения и фамилию задержанного. Однако по  данным  издания ZAXID.NET, это заведующий 1-м хирургическим отделением Львовской областной клинической больницы Ярослав Зборовский. В случае признания судом его виновным врачу грозит лишение свободы на срок от пяти до десяти лет с лишением права занимать определенные должности и с конфискацией имущества. Напомним, ранее правоохранители  задержали  заведующего отделом трансплантации и хирургии печени Национального института трансплантологии им

In [18]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

In [19]:
input_text = dataset_test['text'][INX]

with torch.no_grad():
    tokenized_text = tokenizer(input_text, truncation=True, padding=True, return_tensors='pt')

    source_ids = tokenized_text['input_ids'].to(device, dtype = torch.long)
    source_mask = tokenized_text['attention_mask'].to(device, dtype = torch.long)

    generated_ids = model.generate(
        input_ids = source_ids,
        attention_mask = source_mask, 
        max_length=50,
        num_beams=7,
        temperature = 1.3,
        repetition_penalty=1, 
        length_penalty=1, 
        early_stopping=True,
        no_repeat_ngram_size=2
    )

    pred = tokenizer.decode(generated_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

print("\noutput:\n" + pred)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



output:
Правоохранители задержали руководителя одного из медицинских учреждений Львова. Медик обещал «договориться с руководством районного военного комиссариата об освобождении от прохождения срочной
